In [1]:
import xml.etree.ElementTree as ET
import xmlschema
import subprocess
from datetime import datetime
import re

In [2]:
xml_file_path = 'attachment.xml'
xsd_file_path = 'dcc.xsd'
latex_file = 'final.tex'

In [3]:
try:
    schema = xmlschema.XMLSchema(xsd_file_path)
    schema.validate(xml_file_path)
    print("Validation successful: The XML is valid against the XSD.")
except xmlschema.XMLSchemaValidationError as e:
    print("Validation failed: The XML is not valid against the XSD.")
    print("Validation errors:")
    for error in e.errors:
        print(error)
except Exception as e:
    print("An error occurred:", str(e))

In [4]:
dcc = "{https://ptb.de/dcc}"
si = "{https://ptb.de/si}"
today = datetime.today()

# Parse XML file and extract data
tree = ET.parse(r'attachment.xml')
root = tree.getroot()

# extract other required data
certificate_number = root.find(f'.//{dcc}uniqueIdentifier').text
next_recalibration_date = root.find(f'.//{dcc}statement/{dcc}date').text

customer_name = root.find(f'.//{dcc}customer/{dcc}name/{dcc}content').text
customer_street = root.find(f'.//{dcc}calibrationLaboratory/{dcc}contact/{dcc}location/{dcc}street').text
customer_city = root.find(f'.//{dcc}calibrationLaboratory/{dcc}contact/{dcc}location/{dcc}city').text
customer_postal_code = root.find(f'.//{dcc}calibrationLaboratory/{dcc}contact/{dcc}location/{dcc}postCode').text
customer_reference = root.find(f'.//{dcc}customer/{dcc}location/{dcc}further/{dcc}content').text

item_name = root.find(f'.//{dcc}items/{dcc}item/{dcc}name/{dcc}content').text
model_no = root.find(f'.//{dcc}items/{dcc}item/{dcc}model').text
serial_no = root.find(f'.//{dcc}items/{dcc}item/{dcc}identifications/{dcc}identification/{dcc}name/{dcc}content').text

temperature = root.find(f'.//{dcc}influenceConditions/{dcc}influenceCondition/{dcc}name/{dcc}content[2]').text
temperature = '$'+ temperature.replace('±', '\pm') +'$'

min_humidity = root.find(f'.//{dcc}influenceConditions/{dcc}influenceCondition[2]/{dcc}data/{dcc}quantity/{si}real/{si}value').text
min_humidity_unit = root.find(f'.//{dcc}influenceConditions/{dcc}influenceCondition[2]/{dcc}data/{dcc}quantity/{si}real/{si}unit').text
min_humidity_unit = min_humidity_unit.replace('\\', '\\textbackslash ')

max_humidity = root.find(f'.//{dcc}influenceConditions/{dcc}influenceCondition[2]/{dcc}data/{dcc}quantity[2]/{si}real/{si}value').text
max_humidity_unit = root.find(f'.//{dcc}influenceConditions/{dcc}influenceCondition[2]/{dcc}data/{dcc}quantity[2]/{si}real/{si}unit').text
max_humidity_unit = max_humidity_unit.replace('\\', '\\textbackslash ')

equipment_type = root.find(f'.//{dcc}measuringEquipments/{dcc}measuringEquipment/{dcc}name/{dcc}content').text
associated_uncertainity = root.find(f'.//{dcc}measuringEquipments/{dcc}measuringEquipment/{dcc}identifications/{dcc}identification/{dcc}value').text

head_cft = root.find(f'.//{dcc}respPersons/{dcc}respPerson/{dcc}person/{dcc}name/{dcc}content').text
calibrated_by = root.find(f'.//{dcc}respPersons/{dcc}respPerson[2]/{dcc}person/{dcc}name/{dcc}content').text
checked_by = root.find(f'.//{dcc}respPersons/{dcc}respPerson[3]/{dcc}person/{dcc}name/{dcc}content').text
scientist_incharge = root.find(f'.//{dcc}respPersons/{dcc}respPerson[4]/{dcc}person/{dcc}name/{dcc}content').text

measuring_result = root.find(f'.//{dcc}data/{dcc}list/{dcc}quantity/{si}realListXMLList/{si}valueXMLList').text
measuring_value_probe = root.find(f'.//{dcc}data/{dcc}list/{dcc}quantity[2]/{si}hybrid/{si}realListXMLList/{si}valueXMLList').text
measuring_result_error = root.find(f'.//{dcc}data/{dcc}list/{dcc}quantity[3]/{si}realListXMLList/{si}expandedUncXMLList/{si}uncertaintyXMLList').text
coverage_factor = root.find(f'.//{dcc}data/{dcc}list/{dcc}quantity[3]/{si}realListXMLList/{si}expandedUncXMLList/{si}coverageFactorXMLList').text
coverage_probablity = root.find(f'.//{dcc}data/{dcc}list/{dcc}quantity[3]/{si}realListXMLList/{si}expandedUncXMLList/{si}coverageProbabilityXMLList').text
distribution_type = root.find(f'.//{dcc}data/{dcc}list/{dcc}quantity[3]/{si}realListXMLList/{si}expandedUncXMLList/{si}distributionXMLList').text

begin_calibration_date = root.find(f'.//{dcc}coreData/{dcc}beginPerformanceDate').text
end_calibration_date = root.find(f'.//{dcc}coreData/{dcc}endPerformanceDate').text

coverage_probablity = float(coverage_probablity)
coverage_probablity = 100*coverage_probablity

int_max_humidity = int(max_humidity)
int_min_humidity = int(min_humidity)
diff = (int_max_humidity + int_min_humidity) / 2
diff1 = (diff - int_min_humidity)

string_unit_diff = str(diff) if diff % 1 != 0 else str(int(diff))
string_error_diff = str(diff1) if diff1 % 1 != 0 else str(int(diff1))

humidity_final = '$' + string_unit_diff + '\pm' + string_error_diff + '$'
humidity_unit = '%'

In [5]:
# Split the string by spaces
def change(str):
    split_values = str.split()

    # Process split values to handle strings containing spaces
    processed_values = []
    current = ""

    for val in split_values:
        if val.startswith('\\'):
            # Add the escaped value to the current string
            current += f" {val}"
        else:
            # Add the previous accumulated value if any, then reset current
            if current:
                processed_values.append(current.strip())
                current = ""
            processed_values.append(val)

    # Add the last accumulated value if not already added
    if current:
        processed_values.append(current.strip())

    return processed_values

measuring_result_list = change(measuring_result)
measuring_value_probe_list = change(measuring_value_probe)
measuring_result_error_list = change(measuring_result_error)

In [6]:
def generate_latex_table_with_headline(lists):
    num_columns = len(lists)
    num_rows = max(len(lst) for lst in lists)

    headline = ["Nominal Value (in V)", "Measured Value(in V)", "Error(in V)"]

    table = "\\begin{tabular}{|" + "|".join(["c"] * num_columns) + "|}\n"
    table += "\\hline\n"
    
    # Add headline
    table += " & ".join(headline) + " \\\\\n"
    table += "\\hline\n"

    for i in range(num_rows):
        row = [lst[i] if i < len(lst) else '' for lst in lists]
        table += " & ".join(row) + " \\\\\n"
        table += "\\hline\n"

    table += "\\end{tabular}"
    return table

latex_table = generate_latex_table_with_headline([measuring_result_list, measuring_value_probe_list, measuring_result_error_list])

In [7]:
latex_content = f'''
\\documentclass[a4paper]{{article}}

% Load necessary packages for using Arial-like font
\\usepackage[a-3u]{{pdfx}}
\\usepackage{{attachfile}}
\\usepackage{{geometry}}
\\usepackage{{fancyhdr}}
\\usepackage{{graphicx}}
\\usepackage{{color}}
\\usepackage{{helvet}}  % Helvetica clone
\\usepackage{{embedfile}}
\\usepackage{{tabularx}} % For the tabularx environment
\\usepackage{{siunitx}}
\\usepackage{{lastpage}} % For typesetting units
% Set Arial-like font as the default sans-serif font
\\renewcommand{{\\familydefault}}{{\\sfdefault}}

\\fancyhf{{}}
\\fancyhead[L]{{\\includegraphics[width=0.18\\textwidth,height=25.0mm]{{logo.png}} \\\\[3pt]}}
\\fancyhead[R]{{
    Date: {{\\today}} \\\\
    Next Calibration Date: {next_recalibration_date} \\\\
    {{Page \\thepage}} \\\\
    Certificate Number: {certificate_number} \\\\[2pt]
}}
\\fancyfoot[L]{{
    \\setlength{{\\tabcolsep}}{{1pt}} % Adjusting column separation
        \\begin{{tabularx}}{{\\textwidth}}{{@{{\\extracolsep{{0.001pt}}}} p{{0.1\\textwidth}} X X X X X X X X}}  % Adjusting column widths
            Head CFT: & & Calibrated By: & & Checked By: & & Scientist incharge: & \\\\
            & ({head_cft}) & & ({calibrated_by}) & & ({checked_by}) & & ({scientist_incharge}) \\\\
        \\end{{tabularx}}
        \\vspace{{0.1 cm}}
}} 

\\setlength{{\\headheight}}{{25mm}}
\\pagestyle{{fancy}}
\\renewcommand{{\\headrulewidth}}{{0.8pt}}
\\renewcommand{{\\footrulewidth}}{{0.8pt}}
\\renewcommand{{\\familydefault}}{{\\sfdefault}}
\\newcommand{{\\embedmyfile}}[1]{{\\embedfile{{#1}}}}

\\begin{{document}}
    \\embedfile{{attachment.xml}}
    \\begin{{center}}
        {{\\fontfamily{{phv}}\\fontsize{{18pt}}{{20pt}}\\selectfont\\textbf{{\\textcolor{{black}}{{\\\\[3pt]Calibration Certificate : {item_name}}}}}}}
    \\end{{center}}
    
    \\vspace{{2cm}}  % Adding some vertical space
    
    \\renewcommand{{\\arraystretch}}{{1.5}} % Increasing row height
    
    \\begin{{center}}
        \\setlength{{\\tabcolsep}}{{12pt}} % Adjusting column separation
        \\begin{{tabularx}}{{\\textwidth}}{{@{{\\extracolsep{{8pt}}}} p{{0.4\\textwidth}} X}}  % Adjusting column widths
            1. Calibrated for: & {customer_name} \\\\
            & {customer_street} \\\\
            & {customer_city} \\\\
            & {customer_postal_code} \\\\
            & {customer_reference} \\\\
            &  \\\\
            2. Description and Identification of instrument: & {item_name} \\\\
            & {model_no} \\\\
            & {serial_no} \\\\
            & \\\\
            3. Environmental Conditions:
            & Temperature: {{{temperature}}} \\\\
            & Humidity: ({{{humidity_final}}})\\% \\\\ 
            & \\\\
            4. Standard(s) Used: & {equipment_type} \\\\
            Associated Uncertainity: & {associated_uncertainity} \\\\
            & \\\\
            5. Traceability of standard(s) used: & {equipment_type} (Primary Standard)\\\\
            & \\\\
            6. Principle/ Methodology of Calibration: & The {item_name} has been calibrated by comarison method with {equipment_type} as per calibration procedure no. \\\\
        \\end{{tabularx}}
    \\end{{center}}

    \\pagebreak
    7. Measurements:   \\\\
    \\begin{{center}}
        {latex_table} \\\\
    \\vspace{{0.5 cm}}  
    \\end{{center}}
    The report expanded uncertainity is at a coverage factor k + {coverage_factor} which corresponds approximaltely {coverage_probablity}\% for a {distribution_type} distribution. \\\\
    \\\\

    \\begin{{flushleft}}
        \\setlength{{\\tabcolsep}}{{1pt}} % Adjusting column separation
        \\begin{{tabularx}}{{\\textwidth}}{{@{{\\extracolsep{{8pt}}}} p{{0.4\\textwidth}} X}}  % Adjusting column widths
            8. Dates of Calibration: & {begin_calibration_date} to {end_calibration_date} \\\\
            & \\\\
            9. Remarks: & (i) The {item_name} has been calibrated \\\\
            & (ii) The noise of the {item_name} is inclusive 
        \\end{{tabularx}}
        \\vspace{{2 cm}}
    \\end{{flushleft}}
\\end{{document}}
'''

with open(latex_file, 'w') as f:
    f.write(latex_content)


In [8]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

output_pdf = f'CSIR_{current_datetime}.pdf'

subprocess.run(['pdflatex', '-jobname=' + output_pdf, latex_file])

CompletedProcess(args=['pdflatex', '-jobname=CSIR_2023-12-27_15-12-14.pdf', 'final.tex'], returncode=0)